In [1]:
import os
import pickle
import pandas as pd
import numpy as np

from src.io import read_pkl
from src.default_paths import path_root
from src.mappings import model_names, task_names

In [2]:
PATH_TO_ADAPTER_MODELS_RESULTS = os.path.join(path_root, "data", "evaluate", "adapter_models")
PATH_TO_CLMBR_TASK_RESULTS = os.path.join(path_root, "data", "evaluate", "clmbr_finetuned")

In [3]:
def list_dir(path: str):
    """get list of file/directory names excluding nb checkpoints"""
    
    return [
        x for x in os.listdir(path)
        if x != ".ipynb_checkpoints"
    ]

def get_bstrp_results(results: list, formatted_string: bool = True):
    """return formatted string of bootstrap results"""
    
    lower, med, upper = (
        np.percentile(results,[2.5, 50, 97.5])
        .round(3)
    )
    
    if formatted_string:
        return f"{str(med)} [{str(lower)}, {str(upper)}]"
    else:
        return (lower, med, upper)

In [4]:
use_boot = True

auroc_results = {}
auprc_results = {}

models = list_dir(PATH_TO_ADAPTER_MODELS_RESULTS)

for model in models:
    tasks = list_dir(os.path.join(PATH_TO_ADAPTER_MODELS_RESULTS, model))
    auroc_results[model] = {}
    auprc_results[model] = {}
    
    for task in tasks:
        result = read_pkl(os.path.join(PATH_TO_ADAPTER_MODELS_RESULTS, model, task, "results.pkl"))
        auroc_results[model][task] = get_bstrp_results(result["auroc_bootstrap"]) if use_boot else result["auroc"]
        auprc_results[model][task] = get_bstrp_results(result["auprc_bootstrap"]) if use_boot else result["auprc"]

models = list_dir(PATH_TO_CLMBR_TASK_RESULTS)

for model in models:
    tasks = list_dir(os.path.join(PATH_TO_CLMBR_TASK_RESULTS, model))
    auroc_results[model+"_ft"] = {}
    auprc_results[model+"_ft"] = {}
    
    for task in tasks:
        result = read_pkl(os.path.join(PATH_TO_CLMBR_TASK_RESULTS, model, task, "results.pkl"))
        auroc_results[model+"_ft"][task] =  get_bstrp_results(result["auroc_bootstrap"]) if use_boot else result["auroc"]
        auprc_results[model+"_ft"][task] = get_bstrp_results(result["auprc_bootstrap"]) if use_boot else result["auprc"]

In [5]:
column_orders = [
    "SK [Count]", "SK [Count] (GBM)", 
    "SK", "SK_FT",
    "Stanford", "Stanford_FT",
    "Stanford_Cont-Pre", "Stanford_Cont-Pre_FT"
]

task_orders = [
    "In-hospital Mortality", "Long LOS", "30-day Readmission", 
    #"Hypoglycemia", "Hyperkalemia", "Hyponatremia",
    #"Thrombocytopenia", "Anemia"
]

In [6]:
df_auroc = (
    pd.DataFrame.from_dict(auroc_results)
    .round(3)
    .rename(columns=model_names)
    .reset_index(names="Task")
    .replace(task_names)
    .set_index("Task")
    .reindex(task_orders)
    [column_orders]
)

df_auroc

,SK [Count],SK [Count] (GBM),SK,SK_FT,Stanford,Stanford_FT,Stanford_Cont-Pre,Stanford_Cont-Pre_FT
Task,,,,,,,,
In-hospital Mortality,"0.816 [0.703, 0.906]","0.936 [0.878, 0.975]","0.935 [0.886, 0.972]","0.928 [0.875, 0.968]","0.925 [0.887, 0.957]","0.918 [0.854, 0.96]","0.927 [0.88, 0.963]","0.914 [0.837, 0.963]"
Long LOS,"0.838 [0.822, 0.854]","0.888 [0.876, 0.9]","0.864 [0.851, 0.877]","0.868 [0.855, 0.881]","0.845 [0.83, 0.859]","0.855 [0.841, 0.869]","0.854 [0.839, 0.868]","0.864 [0.85, 0.878]"
30-day Readmission,"0.721 [0.679, 0.761]","0.839 [0.809, 0.867]","0.816 [0.78, 0.848]","0.817 [0.782, 0.849]","0.786 [0.748, 0.822]","0.794 [0.759, 0.828]","0.813 [0.778, 0.846]","0.814 [0.779, 0.847]"


In [7]:
df_auprc = (
    pd.DataFrame.from_dict(auprc_results)
    .round(3)
    .rename(columns=model_names)
    .reset_index(names="Task")
    .replace(task_names)
    .set_index("Task")
    .reindex(task_orders)
    [column_orders]
)

df_auprc

,SK [Count],SK [Count] (GBM),SK,SK_FT,Stanford,Stanford_FT,Stanford_Cont-Pre,Stanford_Cont-Pre_FT
Task,,,,,,,,
In-hospital Mortality,"0.035 [0.015, 0.067]","0.119 [0.052, 0.237]","0.209 [0.065, 0.394]","0.167 [0.051, 0.338]","0.094 [0.035, 0.224]","0.122 [0.038, 0.279]","0.216 [0.051, 0.403]","0.151 [0.045, 0.319]"
Long LOS,"0.607 [0.569, 0.645]","0.716 [0.683, 0.748]","0.64 [0.602, 0.677]","0.656 [0.618, 0.691]","0.619 [0.581, 0.654]","0.63 [0.592, 0.668]","0.636 [0.599, 0.671]","0.663 [0.625, 0.698]"
30-day Readmission,"0.172 [0.134, 0.216]","0.402 [0.329, 0.474]","0.316 [0.249, 0.385]","0.317 [0.251, 0.385]","0.246 [0.192, 0.307]","0.267 [0.207, 0.332]","0.301 [0.237, 0.366]","0.319 [0.253, 0.384]"


In [10]:
if use_boot:
    df_auroc.to_csv("../results/main_auroc_boot.csv")
    df_auprc.to_csv("../results/main_auprc_boot.csv")
else:
    df_auroc.to_csv("../results/main_auroc.csv")
    df_auprc.to_csv("../results/main_auprc.csv")